# Homework 3-Question 1:  Doodle scheduling
Doodle Inc. is looking to interview a candidate for a new software
engineer position at their company. It works like this: the interview (10 AM to 3 PM) is divided into
a number of 20-minute time slots that may be used for 1-on-1 meetings with the candidate. There is
also a one-hour time slot in the middle of the day where 3 employees take the candidate out for lunch.
It would be nice for all 15 senior employees to meet with the candidate at some point during the
day, but everybody has a busy schedule so it’s not clear whether this will be possible. A doodle poll
(obviously) was sent to the 15 senior employees to figure out their availability

 | | 10:00 | 10:20 | 10:40 | 11:00 | 11:20 | 11:40 | Lunch | 1:00 | 1:20 | 1:40 | 2:00 | 2:20 | 2:40
 | ---| ---| ---| ---| ---| ---| ---| ---| ---| ---| ---| ---| ---| ---| --- 
 | Manuel | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0
 | Luca | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0
 | Jule | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1
 | Michael | 0 | 0 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 0
 | Malte | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0
 | Chris | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0
 | Spyros | 0 | 0 | 0 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
 | Mirjam | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 1
 | Matt | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0
 | Florian | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0
 | Josep | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0
 | Joel | 1 | 1 | 0 | 0 | 0 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 1
 | Tom | 1 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 1 | 1
 | Daniel | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0
 | Anne | 1 | 1 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0
 
 In the table, a 1 means that the employee is available at the indicated time, while a 0 means that they
are unavailable. Determine whether a feasible interview schedule exists. If so, print out a calendar for
the candidate that lists who they will be meeting at each time slot.

## Problem Data

In [1]:
using NamedArrays
# Employees
employees = ["Manuel", "Luca", "Jule", "Michael", "Malte", "Chris", 
    "Spyros", "Mirjam", "Matt", "Florian", "Josep", "Joel", "Tom", "Daniel", "Anne"]

# Slots
slots = ["10:00","10:20","10:40","11:00","11:20","11:40","Lunch",
    "1:00","1:20","1:40","2:00","2:20","2:40"]

# Slot Capacity
slotCapacity = Dict(zip(slots,[1,1,1,1,1,1,3,1,1,1,1,1,1]))

raw = [ 0 0 1 1 0 0 0 1 1 0 0 0 0;
        0 1 1 0 0 0 0 0 1 1 0 0 0;
        0 0 0 1 1 0 1 1 0 1 1 1 1;
        0 0 0 1 1 1 1 1 1 1 1 1 0;
        0 0 0 0 0 0 1 1 1 0 0 0 0;
        0 1 1 0 0 0 0 0 1 1 0 0 0;
        0 0 0 1 1 1 1 0 0 0 0 0 0;
        1 1 0 0 0 0 0 0 0 0 1 1 1;
        1 1 1 0 0 0 0 0 0 1 1 0 0;
        0 0 0 0 0 0 0 1 1 0 0 0 0;
        0 0 0 0 0 0 1 1 1 0 0 0 0;
        1 1 0 0 0 1 1 1 1 0 0 1 1;
        1 1 1 0 1 1 0 0 0 0 0 1 1;
        0 1 1 1 0 0 0 0 0 0 0 0 0;
        1 1 0 0 1 1 0 0 0 0 0 0 0;]

availability = NamedArray(raw, (employees, slots), ("Employees", "Slots"));

## Problem Model 

In [2]:
using JuMP

m = Model()

# decision variables, flow on edges from Layer 1: employees to Layer 2: slots
@variable(m, x[employees,slots] >= 0)

# availability constraint, to assign an employee to slot only when the employee is available.
for e in employees
    for s in slots
        @constraint(m, x[e,s] <= availability[e,s])
    end
end

# constraint to limit only one employee per slot and three employees for lunch.
@constraint(m,attendancePerSlot[s in slots],sum(x[e,s] for e in employees) <= slotCapacity[s])

# constraint for all employees to meet the candidate exactly once.
@constraint(m,attendancePerEmployee[e in employees],sum(x[e,s] for s in slots) == 1)

# Max flow objective along edges
@objective(m, Max, sum(x[e,s] for e in employees for s in slots))
;

In [3]:
status = solve(m)
println(status)
println(getobjectivevalue(m))
# nicely formatted solution
solution = NamedArray( Int[getvalue(x[i,j]) for i in employees, j in slots], (employees,slots),
    ("Employees","Slots") )
[println(s,"\t:\t" ,e) for s in slots for e in employees if solution[e,s] == 1]
println()

Optimal
15.0
10:00	:	Mirjam
10:20	:	Joel
10:40	:	Manuel
11:00	:	Daniel
11:20	:	Jule
11:40	:	Anne
Lunch	:	Malte
Lunch	:	Spyros
Lunch	:	Josep
1:00	:	Florian
1:20	:	Luca
1:40	:	Chris
2:00	:	Matt
2:20	:	Michael
2:40	:	Tom



The assignment of employees to slots has been shown above.